In [3]:
df = spark.read.option("multiline", "true").json("Files/bing_news.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing_news.json".
display(df)

StatementMeta(, 9c446857-be5e-44a8-905e-54eef3cd5885, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c5b38e23-1e84-40c8-b3c6-a39bf9757457)

In [4]:
df1=df.select('value')

StatementMeta(, 9c446857-be5e-44a8-905e-54eef3cd5885, 6, Finished, Available, Finished)

In [5]:
from pyspark.sql.functions import explode,col

df_exploded = df1.select(explode(col("value")).alias("json_object"))

display(df_exploded)


StatementMeta(, 9c446857-be5e-44a8-905e-54eef3cd5885, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0273eefd-a1d7-41bf-a48c-90b0027e5cda)

In [6]:
json_list=df_exploded.toJSON().collect()

print(json_list)

StatementMeta(, 9c446857-be5e-44a8-905e-54eef3cd5885, 8, Finished, Available, Finished)

['{"json_object":{"about":[{"name":"Karnataka","readLink":"https://api.bing.microsoft.com/api/v7/entities/216903eb-bbc1-497e-b914-8eb69db6f747"},{"name":"Karnataka High Court","readLink":"https://api.bing.microsoft.com/api/v7/entities/79e8a2f6-8792-bda6-c3f0-762f3f58c117"},{"name":"Siddaramaiah","readLink":"https://api.bing.microsoft.com/api/v7/entities/2b87b63a-0264-3d22-fabd-9598bf331d45"},{"name":"Bengaluru FC","readLink":"https://api.bing.microsoft.com/api/v7/entities/e3027fb3-3f4b-4c71-e40c-e07ddcf89bd3"},{"name":"D. K. Shivakumar","readLink":"https://api.bing.microsoft.com/api/v7/entities/9e4fedf7-2f89-4e4a-a798-5dc3859eb63b"},{"name":"Bharatiya Janata Party","readLink":"https://api.bing.microsoft.com/api/v7/entities/487bfae9-1de7-98a0-5eec-cfb87bfd5c0a"}],"category":"World","datePublished":"2025-06-05T17:43:00.0000000Z","description":"Karnataka Deputy CM DK Shivakumar broke down in tears while speaking to reporters about the deaths of teenagers in the stampede on Wednesday. The 

In [7]:
import json

news_json=json.loads(json_list[0])

print(news_json)


StatementMeta(, 9c446857-be5e-44a8-905e-54eef3cd5885, 9, Finished, Available, Finished)

{'json_object': {'about': [{'name': 'Karnataka', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/216903eb-bbc1-497e-b914-8eb69db6f747'}, {'name': 'Karnataka High Court', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/79e8a2f6-8792-bda6-c3f0-762f3f58c117'}, {'name': 'Siddaramaiah', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/2b87b63a-0264-3d22-fabd-9598bf331d45'}, {'name': 'Bengaluru FC', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/e3027fb3-3f4b-4c71-e40c-e07ddcf89bd3'}, {'name': 'D. K. Shivakumar', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/9e4fedf7-2f89-4e4a-a798-5dc3859eb63b'}, {'name': 'Bharatiya Janata Party', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/487bfae9-1de7-98a0-5eec-cfb87bfd5c0a'}], 'category': 'World', 'datePublished': '2025-06-05T17:43:00.0000000Z', 'description': 'Karnataka Deputy CM DK Shivakumar broke down in tears while speaking to reporters about the deaths of teenagers in th

In [8]:
print(news_json['json_object']['name'])
print(news_json['json_object']['description'])
print(news_json['json_object']['category'])
print(news_json['json_object']['image']['thumbnail']['contentUrl'])
print(news_json['json_object']['url'])
print(news_json['json_object']['provider'][0]['name'])
print(news_json['json_object']['datePublished'])

StatementMeta(, 9c446857-be5e-44a8-905e-54eef3cd5885, 10, Finished, Available, Finished)

Bengaluru Stampede News Live Updates: New city police commissioner appointed day after deadly stampede
Karnataka Deputy CM DK Shivakumar broke down in tears while speaking to reporters about the deaths of teenagers in the stampede on Wednesday. The Congress leader said that no one can understand the pain of the families,
World
https://www.bing.com/th?id=OVFT.JaSiloJyNlY3wfLOr0NUdC&pid=News
https://indianexpress.com/article/cities/bangalore/bengaluru-stampede-news-live-updates-ipl-rcb-victory-march-chinnaswamy-stadium-10049263/
The Indian Express
2025-06-05T17:43:00.0000000Z


In [9]:
import json

name=[]
description=[]
category=[]
url=[]
image=[]
provider=[]
datePublished=[]

for json_str in json_list:
    try:
        article=json.loads(json_str)
        if article['json_object'].get('category') and article['json_object'].get('image',{}).get('thumbnail',{}).get('contentUrl'):

            name.append(article['json_object']['name'])
            description.append(article['json_object']['description'])
            category.append(article['json_object']['category'])
            url.append(article['json_object']['url'])
            image.append(article['json_object']['image']['thumbnail']['contentUrl'])
            provider.append(article['json_object']['provider'][0]['name'])
            datePublished.append(article['json_object']['datePublished'])

    except Exception as e:
        print(f"Error processing JSON object: {e}")

StatementMeta(, 9c446857-be5e-44a8-905e-54eef3cd5885, 11, Finished, Available, Finished)

In [10]:
from pyspark.sql.types import StructType, StructField, StringType


# Combine the lists
data = list(zip(name,description,category,url,image,provider,datePublished))

# Define schema
schema = StructType([
    StructField("name", StringType(), True),
    StructField("description", StringType(), True),
    StructField("category", StringType(), True),
    StructField("url", StringType(), True),
    StructField("image", StringType(), True),
    StructField("provider", StringType(), True),
    StructField("datePublished", StringType(), True)
])

# Create DataFrame
df_cleaned = spark.createDataFrame(data, schema=schema)

StatementMeta(, 9c446857-be5e-44a8-905e-54eef3cd5885, 12, Finished, Available, Finished)

In [11]:
display(df_cleaned)

StatementMeta(, 9c446857-be5e-44a8-905e-54eef3cd5885, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 04b41d5f-7dc6-4ea5-bfb7-5d8729aaaeb4)

In [12]:
from pyspark.sql.functions import to_date,date_format

df_cleaned_final=df_cleaned.withColumn('datePublished',date_format(to_date('datePublished'),'dd-MMM-yyyy'))

#df_cleaned_final = df_cleaned.withColumn("datePublished", date_format(to_date("datePublished"), "dd-MMM-yyyy"))


display(df_cleaned_final)

StatementMeta(, 9c446857-be5e-44a8-905e-54eef3cd5885, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, eb92a38b-da13-4efe-9290-c8e82526c436)

In [13]:
from pyspark.sql.functions import to_date, date_format

df_cleaned_final = df_cleaned.withColumn("datePublished", date_format(to_date("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, 9c446857-be5e-44a8-905e-54eef3cd5885, 15, Finished, Available, Finished)

In [15]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'Bing_LH.tbl_latest_news'

    df_cleaned_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    df_cleaned_final.createOrReplaceTempView("vw_df_cleaned_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_df_cleaned_final source_view

                    ON source_view.url = target_table.url
                    
                    WHEN MATCHED AND 
                    source_view.name <> target_table.name OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished   

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *

                """)

StatementMeta(, 9c446857-be5e-44a8-905e-54eef3cd5885, 17, Finished, Available, Finished)